In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve

# Step 1: Load and Preprocess Data

In [3]:
def load_data_from_psv(folder_path):
    """Load all .psv files and concatenate into a single DataFrame."""
    all_files = glob.glob(os.path.join(folder_path, '*.psv'))
    data_list = []
    for file in all_files:
        df = pd.read_csv(file, sep='|')
        data_list.append(df)
    return pd.concat(data_list, ignore_index=True)

data_folder = r'C:\Users\uSER\source\repos\alternative-assignment-aml\training_setA\training'
data = load_data_from_psv(data_folder)

# Step 2: Handle Missing Values

In [4]:
def preprocess_data(df):
    """Preprocess the data: handle missing values and normalize."""
    # Fill missing values (forward fill, then backward fill as fallback)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    # Normalize continuous features
    scaler = StandardScaler()
    continuous_features = [col for col in df.columns if col != 'SepsisLabel' and df[col].dtype in [np.float64, np.int64]]
    df[continuous_features] = scaler.fit_transform(df[continuous_features])

    return df, continuous_features

# Preprocess the data
data, feature_columns = preprocess_data(data)

C:\Users\uSER\AppData\Local\Temp\ipykernel_28152\1711906078.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\uSER\AppData\Local\Temp\ipykernel_28152\1711906078.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
c:\Users\uSER\anaconda3\envs\aaa_env\lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\uSER\anaconda3\envs\aaa_env\lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\uSER\anaconda3\envs\aaa_env\lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  ne

# Step 3: Prepare Data for LSTM

In [5]:
def create_sequences_optimized(df, features, target, seq_length):
    """Efficiently create sequences of data for LSTM input."""
    data_array = df[features].values  # Convert to NumPy array
    target_array = df[target].values  # Convert target to NumPy array

    # Ensure sufficient data for sequences
    if len(data_array) <= seq_length:
        raise ValueError("Data length must be greater than the sequence length.")

    num_samples = len(data_array) - seq_length + 1  # Total sequences
    X = np.lib.stride_tricks.sliding_window_view(data_array, (seq_length, len(features)))
    X = X.reshape(num_samples, seq_length, len(features))  # Adjust shape

    y = target_array[seq_length - 1:]  # Align target labels

    return X, y

# Define sequence length and prepare data
sequence_length = 10
X, y = create_sequences_optimized(data, feature_columns, 'SepsisLabel', sequence_length)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Step 4: Build LSTM Model

In [6]:
def build_lstm_model(input_shape):
    """Build and compile an LSTM model."""
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True, activation='tanh'),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(64, activation='tanh'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)

# Step 5: Train the Model

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 86s 10ms/step - accuracy: 0.9782 - loss: 0.3178 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 2/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.9786 - loss: 0.1036 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 3/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.9783 - loss: 0.1047 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 4/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.9783 - loss: 0.1047 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 5/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.9783 - loss: 0.1047 - val_accuracy: 0.9783 - val_loss: 0.1045
Epoch 6/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.9782 - loss: 0.1050 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 7/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.9783 - loss: 0.1047 - val_accuracy: 0.9783 - val_loss: 0.1046
Epoch 8/20
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 82s 10ms/step - accuracy: 0.9781 - loss:

# Step 6: Evaluate the Model

In [8]:
def evaluate_model(model, X_test, y_test):
    """Evaluate the LSTM model."""
    loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

evaluate_model(model, X_test, y_test)

3705/3705 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9785 - loss: 0.1040
Test Loss: 0.10451813042163849, Test Accuracy: 0.97831791639328


# Step 7: Optimize Classification Threshold

In [9]:
preds = model.predict(X_val)
precision, recall, thresholds = precision_recall_curve(y_val, preds)
optimal_threshold = thresholds[np.argmax(precision * recall)]
print(f"Optimal Threshold: {optimal_threshold}")

3705/3705 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step
Optimal Threshold: 0.02134244330227375


# Step 8: Save the Model

In [10]:
model.save('optimized_sepsis_lstm_model.h5')

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Lambda, Layer

In [19]:
# Build Variational Autoencoder (VAE)

# Define VAE Encoder
def build_vae_encoder(input_dim, latent_dim):
    inputs = Input(shape=(input_dim,))
    h = Dense(64, activation='relu')(inputs)
    z_mean = Dense(latent_dim, name='z_mean')(h)
    z_log_var = Dense(latent_dim, name='z_log_var')(h)
    return Model(inputs, [z_mean, z_log_var], name="encoder"), inputs

# Define Sampling Layer
class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define VAE Decoder
def build_vae_decoder(latent_dim, output_dim):
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(64, activation='relu')(latent_inputs)
    outputs = Dense(output_dim, activation='sigmoid')(x)
    return Model(latent_inputs, outputs, name="decoder")



In [39]:

# VAE Loss Layer Integration
class VAELossLayer(Layer):
    def call(self, inputs):
        vae_inputs, vae_outputs, z_mean, z_log_var = inputs
        # Reconstruction Loss
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(tf.square(vae_inputs - vae_outputs), axis=-1)
        )
        # KL Divergence Loss
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        )
        total_loss = reconstruction_loss + kl_loss
        self.add_loss(total_loss)
        return vae_outputs  # Pass outputs for further us



In [40]:
# Replace VAE Model Definition
vae_inputs_layer = Input(shape=(vae_input_dim,))
vae_encoder_outputs = encoder(vae_inputs_layer)
z_mean, z_log_var = vae_encoder_outputs
z = Sampling()([z_mean, z_log_var])
vae_decoder_outputs = decoder(z)

In [41]:
# Add Loss Computation Layer
vae_outputs = VAELossLayer()([vae_inputs_layer, vae_decoder_outputs, z_mean, z_log_var])

# Final VAE Model
vae = Model(vae_inputs_layer, vae_outputs, name='vae')

In [42]:
# Compile and Train the VAE
vae.compile(optimizer='adam')
vae.fit(X_train[:, -1, :], X_train[:, -1, :], epochs=50, batch_size=64, validation_data=(X_val[:, -1, :], X_val[:, -1, :]))



Epoch 1/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - loss: nan - val_loss: nan
Epoch 2/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 3/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 4/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 5/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 6/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 7/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 8/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 9/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 10/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: nan - val_loss: nan
Epoch 11/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: nan - val_loss: nan
Epoch 12/50
8643/8643 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: nan - val_loss: nan
E

In [43]:
# Generate Personalized Recommendations
def generate_recommendations(vae, patient_data):
    """Generate personalized recommendations using the VAE model."""
    synthetic_pathway = vae.predict(patient_data)
    recommendations = []
    for pathway in synthetic_pathway:
        if pathway.mean() > 0.5:  # Example condition for recommendations
            recommendations.append("Recommend aggressive intervention.")
        else:
            recommendations.append("Recommend close monitoring.")
    return recommendations



In [44]:
# Example Usage with LSTM + VAE
patient_data = X_test[:, -1, :]
recommendations = generate_recommendations(vae, patient_data)
for i, rec in enumerate(recommendations[:5]):
    print(f"Patient {i + 1}: {rec}")

3705/3705 ━━━━━━━━━━━━━━━━━━━━ 3s 687us/step
Patient 1: Recommend close monitoring.
Patient 2: Recommend close monitoring.
Patient 3: Recommend close monitoring.
Patient 4: Recommend close monitoring.
Patient 5: Recommend close monitoring.
